# Inaugural Project

Imports and set magics:

In [13]:
import numpy as np
import matplotlib.pylab as plt
from scipy import optimize
# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import inauguralproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Question 1

In [14]:
#In this question, we want to construct a function which solves equation (1). 

#Firstly, the utility function is defined by: 
def utility(w, l, m = 1, v = 10, epsilon = 0.3, tau0 = 0.4, tau1 = 0.1, kappa = 0.4):
    tax = tau0*w*l+tau1*np.fmax(w*l-kappa,0)
    c = m+w*l-tax
    utility = np.log(c)-v*l**(1+1/epsilon)/(1+1/epsilon)
    return utility,c, tax

#Next a function is created in order to maximize utility:
def solveconsumerproblem(w, m = 1, v = 10, epsilon = 0.3, tau0 = 0.4, tau1 = 0.1, kappa = 0.4,\
                         N=100, callableoutput=False, tax=False):


#Now, we want to create a set of emply local lists in order to store the outputs. 
    c_star = [0]
    l_star = [0]
    t_star = [0]
    utility_star = [-np.inf]

# Creating a range for labour in order to examine the utility for a given number of elements. 
    Range = np.linspace(0, 1, N)
    for i in Range:
        utility_temp = utility(l = i, w = w, epsilon = epsilon, tau0 = tau0, tau1 = tau1, kappa = kappa)
        if utility_temp[0]>utility_star:
            utility_star[0]=utility_temp[0]
            l_star[0] = i
            c_star[0] = utility_temp[1]
            t_star[0] = utility_temp[2]

#Setting requirements for tax, whether to display or not
    if tax == False:
        if callableoutput == False:
            print(f"It is found that (l*,c*) = ({l_star[0]:.3}, {c_star[0]:.3}) meaning that the optimal utility is given by u(c*,l*) = {utility_star[0]:.3}")  
        else:
            return utility_star[0], l_star[0], c_star[0]
    
    if tax == True:
            return utility_star[0], l_star[0], c_star[0], t_star[0]

solveconsumerproblem(w=0.6, tax=False, callableoutput=False)

It is found that (l*,c*) = (0.354, 1.13) meaning that the optimal utility is given by u(c*,l*) = 0.0943


# Question 2

In [15]:
# code

# Question 3

ADD ANSWER.

In [16]:
# code

# Question 4

ADD ANSWER.

In [17]:
# code

# Question 5

ADD ANSWER.

In [18]:
# code

# Conclusion

ADD CONCISE CONLUSION.